In [1]:
%%time
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl
!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Processing /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl
CPU times: user 2.18 s, sys: 570 ms, total: 2.75 s
Wall time: 3min 41s


In [2]:
!pip install transformers peft accelerate \
    -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [3]:
!pip install pylatexenc -q -U --no-index --find-links /kaggle/input/pylatexenc-wheel

In [4]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import transformers
import vllm
from transformers import AutoTokenizer, AutoModel
import pandas as pd

2024-12-11 13:38:06,076	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import sys
print(sys.path)

['/opt/conda/lib/python3.10/site-packages/ray/thirdparty_files', '/kaggle/lib/kagglegym', '/kaggle/lib', '/kaggle/input/eedi-mining-misconceptions-in-mathematics', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper']


In [6]:
import os
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))
print(IS_SUBMISSION)

False


## Metrics
The metrics that we will use for this problem: MAP@25

In [7]:
%%writefile eedi_metrics.py

# Credit: https://www.kaggle.com/code/abdullahmeda/eedi-map-k-metric

import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold

def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def recall_at_k(actual, predicted, k=25):
    """
    Computes the recall at k, handling cases where the inputs might be numpy arrays or lists.
    
    Parameters
    ----------
    actual : int/float, single element
    predicted : list
                A list of predicted elements (order matters up to k)
    k : int, optional
        The maximum number of predicted elements to consider

    Returns
    -------
    score : float
            The recall at k
    """
    if not actual:
        return 0.0

    if len(predicted) > k:
        predicted = predicted[:k]

    # Convert numpy arrays or lists to tuples if necessary
    actual = int(actual[0])
    predicted = {int(item) if isinstance(item, (np.ndarray, list)) else item for item in predicted}
    # print("Actual: ", actual)
    # print("predicted: ", predicted)
    # print("Actual type: ", type(actual))
    # print("Predicted type: ", type(predicted))
    if actual in predicted:
        return 1
    else:
        return 0

def averaged_recall_at_k(actual, predicted, k=25):
    return np.mean([recall_at_k(a,p,k) for a,p in zip(actual, predicted)])

def dcg_at_k(relevances, k):
    """Compute the Discounted Cumulative Gain at rank k"""
    relevances = np.asfarray(relevances)[:k]
    if relevances.size:
        return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))
    return 0.0

def ndcg_at_k(actual, predicted, k):
    """Compute the Normalized Discounted Cumulative Gain at rank k"""
    if k > len(predicted):
        k = len(predicted)

    # Actual relevance: 1 if the item is the ground truth, 0 otherwise
    actual_relevance = [1 if p == actual else 0 for p in predicted]

    # Compute DCG@k
    dcg_score = dcg_at_k(actual_relevance, k)

    # Compute IDCG@k (best possible DCG@k)
    idcg_score = dcg_at_k([1], k)  # Best case: the relevant item is at the top of the list
    if idcg_score == 0:
        return 0.0
    return dcg_score / idcg_score
def averaged_ndcg_at_k(actual, predicted, k=25):
    return np.mean([ndcg_at_k(a,p,k) for a,p in zip(actual, predicted)])

Writing eedi_metrics.py


## LLM reasoning
Use Qwen2.5 14B Instruct AWQ to generate Reasoning HyDE if necessary. Possible choices:
1. Generate HyDE with LLM misconception zero-shot reasoning
2. Generate HyDE with LLM misconception, with self consistency
3. No reasoning, just reformat the question into a query string

In [8]:
%%writefile run_vllm.py

import os
from transformers import AutoTokenizer
import pandas as pd
import re
import vllm
import pandas as pd
import torch
import multiprocessing
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#clear cache
torch.cuda.empty_cache()
use_online_synthetic_data=False
activate_reasoning = True
SELF_CONSISTENCY = False #only be used if this is a reasoning problem
SELF_CONSISTENCY_AMT=5 #only be used if SELF_CONSISTENCY_AMT is True
#LLM_MODEL_PATH="/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
LLM_MODEL_PATH="/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ"
TRAIN_MODE = False


IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))
if use_online_synthetic_data:
    df_train = pd.read_csv("/flash2/aml/wangwd24/eedi-mining-misconceptions-in-mathematics/eedi_synthetic.csv").fillna(-1)
else:
    df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1)
df_test = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_PATH)
PROMPT_ORIGINAL  = """Question: {Question}
Incorrect Answer: {IncorrectAnswer}
Correct Answer: {CorrectAnswer}
Construct Name: {ConstructName}
Subject Name: {SubjectName}

Your task: You are a genius mathematician. Identify the misconception behind Incorrect Answer. Answer concisely and generically inside <response>$$INSERT TEXT HERE$$</response>.
Before answering the question think step by step concisely in 1-2 sentence inside <thinking>$$INSERT TEXT HERE$$</thinking> tag and respond your final misconception inside <response>$$INSERT TEXT HERE$$</response> tag."""


PROMPT_QUERY_NO_REASON = """Question: {Question}
Incorrect Answer: {IncorrectAnswer}
Correct Answer: {CorrectAnswer}
Construct Name: {ConstructName}
Subject Name: {SubjectName}
"""


def apply_template(row, tokenizer, targetCol, online_synthetic_data=False):
    if not online_synthetic_data:
        messages = [
            {
                "role": "user", 
                "content": PROMPT_ORIGINAL.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"Answer{targetCol}Text"],
                    CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"])
            }
        ]
    else:
        messages = [
            {
                "role": "user", 
                "content": PROMPT_ORIGINAL.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"AnswerText"],
                    CorrectAnswer=row[f"CorrectAnswerText"])
            }
        ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

def apply_template_no_reason(row, targetCol, online_synthetic_data=False):
    if not online_synthetic_data:
        messages = [
            PROMPT_QUERY_NO_REASON.format(
                ConstructName=row["ConstructName"],
                SubjectName=row["SubjectName"],
                Question=row["QuestionText"],
                IncorrectAnswer=row[f"Answer{targetCol}Text"],
                CorrectAnswer=row[f"Answer{row.CorrectAnswer}Text"])
        ]
    else:
        messages = [
            {
                "role": "user", 
                "content": PROMPT_ORIGINAL.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"AnswerText"],
                    CorrectAnswer=row[f"CorrectAnswerText"])
            }
        ]
    return messages

print("Executing: ")
df = {}
question_level_df_data = {} #make a dataframe that maps to each question level
#initialize required column
question_level_df_data["QuestionId_Answer"] = []
question_level_df_data["ConstructName"] = []
question_level_df_data["SubjectName"] = []
question_level_df_data["QuestionText"] = []
question_level_df_data["incorrect_answer"] = []
question_level_df_data["correct_answer"] = []
if TRAIN_MODE:
    print("Is not submission")
    df_label = {}
    for idx, row in df_train.iterrows():
        if not use_online_synthetic_data:
            for option in ["A", "B", "C", "D"]:
                if (row.CorrectAnswer!=option) & (row[f"Misconception{option}Id"]!=-1):
                    #populate question level dataframe
                    question_level_df_data["QuestionId_Answer"].append(f"{row.QuestionId}_{option}")
                    question_level_df_data["ConstructName"].append(row["ConstructName"])
                    question_level_df_data["SubjectName"].append(row["SubjectName"])
                    question_level_df_data["QuestionText"].append(row["QuestionText"])
                    question_level_df_data["incorrect_answer"].append(row[f"Answer{option}Text"])
                    question_level_df_data["correct_answer"].append(row[f"Answer{row.CorrectAnswer}Text"])

                    if activate_reasoning:
                        df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
                    else:
                        df[f"{row.QuestionId}_{option}"] = apply_template_no_reason(row, option)
                    df_label[f"{row.QuestionId}_{option}"] = [row[f"Misconception{option}Id"]]
        else:
            #kaggle online synthetic data
            if activate_reasoning:
                df[f"Synthetic_{idx}"] = apply_template(row, tokenizer, "", online_synthetic_data=True)
            else:
                df[f"Synthetic_{idx}"] = apply_template_no_reason(row, "", online_synthetic_data=True)
            df_label[f"Synthetic_{idx}"] = [row["MisconceptionID"]]
    df_label = pd.DataFrame([df_label]).T.reset_index()
    df_label.columns = ["QuestionId_Answer", "MisconceptionId"]
    df_label.to_parquet("/kaggle/working/label.parquet", index=False)
else:
    print("Entering test mode")
    for idx, row in df_test.iterrows():
        print("Row question id: ", row.QuestionId)
        for option in ["A", "B", "C", "D"]:
            if row.CorrectAnswer!=option:
                #populate question level dataframe
                question_level_df_data["QuestionId_Answer"].append(f"{row.QuestionId}_{option}")
                question_level_df_data["ConstructName"].append(row["ConstructName"])
                question_level_df_data["SubjectName"].append(row["SubjectName"])
                question_level_df_data["QuestionText"].append(row["QuestionText"])
                question_level_df_data["incorrect_answer"].append(row[f"Answer{option}Text"])
                question_level_df_data["correct_answer"].append(row[f"Answer{row.CorrectAnswer}Text"])

                if activate_reasoning:
                    df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer, option)
                else:
                    df[f"{row.QuestionId}_{option}"] = apply_template(row, tokenizer)

df = pd.DataFrame([df]).T.reset_index()
df.columns = ["QuestionId_Answer", "text"]
df.to_parquet("/kaggle/working/temporary.parquet", index=False)

# =========================== Beginning input generation for embedding model ===========================
if TRAIN_MODE:
    if use_online_synthetic_data:
        df_gt_addr = "/flash2/aml/wangwd24/eedi-mining-misconceptions-in-mathematics/eedi_synthetic.csv"
    else:
        df_gt_addr = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv"
        
else:
    df_gt_addr = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv"

df = pd.read_parquet("/kaggle/working/temporary.parquet")
df_train = pd.read_csv(df_gt_addr).fillna(-1)
print("df_train head: ")
print(df_train.head())

PROMPT_SELF_CONSISTENCY = """Question: {Question}
Incorrect Answer: {IncorrectAnswer}
Correct Answer: {CorrectAnswer}
Construct Name: {ConstructName}
Subject Name: {SubjectName}

Your task: You are a genius mathematician. Identify the misconception behind Incorrect Answer. You have multiple reasoning paths behind misconceptions and use self-consitency to choose the accurate one. Answer concisely and generically inside <response>$$INSERT TEXT HERE$$</response>.
Before answering the question think step by step concisely in 1-2 sentence inside <thinking>$$INSERT TEXT HERE$$</thinking> tag and respond your final misconception inside <response>$$INSERT TEXT HERE$$</response> tag.

{ReasoningPaths}
"""

def apply_template_self_consistency(row_from_df, tokenizer, reason_path_responses, targetCol, online_synthetic_data=False):
    reasoning_text = ""
    reasoning_length = len(reason_path_responses)
    for idx in range(reasoning_length):
        letter_idx = chr(ord('A') + idx)
        reasoning_text += f"""Reasoning {letter_idx}
        {reason_path_responses[idx]}
        
        """
    if not online_synthetic_data:
        messages = [
            {
                "role": "user", 
                "content": PROMPT_SELF_CONSISTENCY.format(
                    ConstructName=row_from_df["ConstructName"],
                    SubjectName=row_from_df["SubjectName"],
                    Question=row_from_df["QuestionText"],
                    IncorrectAnswer=row_from_df[f"Answer{targetCol}Text"],
                    CorrectAnswer=row_from_df[f"Answer{row_from_df.CorrectAnswer}Text"],
                    ReasoningPaths=reasoning_text),
            }
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        messages = [
            {
                "role": "user", 
                "content": PROMPT_SELF_CONSISTENCY.format(
                    ConstructName=row_from_df["ConstructName"],
                    SubjectName=row_from_df["SubjectName"],
                    Question=row_from_df["QuestionText"],
                    IncorrectAnswer=row_from_df[f"AnswerText"],
                    CorrectAnswer=row_from_df[f"CorrectAnswerText"],
                    ReasoningPaths=reasoning_text),
            }
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

llm = vllm.LLM(
    LLM_MODEL_PATH,
    quantization="awq",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,#8192
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()
df_new = {}

if activate_reasoning: 
    if SELF_CONSISTENCY:
        print("Self Consistency running!")
        responses_list = []
        for i in range(SELF_CONSISTENCY_AMT):
            responses = llm.generate(
                df["text"].values,
                vllm.SamplingParams(
                    n=1,  # Number of output sequences to return for each prompt.
                    top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
                    temperature=0.1,  # randomness of the sampling
                    seed=777, # Seed for reprodicibility
                    skip_special_tokens=False,  # Whether to skip special tokens in the output.
                    max_tokens=2048,  # Maximum number of tokens to generate per output sequence.
                ),
                use_tqdm = True
            )
            responses = [x.outputs[0].text for x in responses]
            responses_list.append(responses)
        responses_list = list(zip(*responses_list))
        counter = 0
        final_text = []
        for idx, row_from_df in df_train.iterrows():
            if not use_online_synthetic_data:
                for option in ["A", "B", "C", "D"]:
                    if TRAIN_MODE:
                        boolForSelfConsistency = (row_from_df.CorrectAnswer!=option) & (row_from_df[f"Misconception{option}Id"]!=-1)
                    else:
                        boolForSelfConsistency = (row_from_df.CorrectAnswer!=option)
                    if boolForSelfConsistency:
                        temp_text = apply_template_self_consistency(row_from_df, tokenizer, responses_list[counter], option)
                        counter += 1
                        final_text.append(temp_text)
            else:
                temp_text = apply_template_self_consistency(row_from_df, tokenizer, responses_list[counter], "", online_synthetic_data=True)
                counter += 1
                final_text.append(temp_text)


        print("Final text: ", final_text)

        responses = llm.generate(
            final_text,
            vllm.SamplingParams(
                n=1,  # Number of output sequences to return for each prompt.
                top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
                temperature=0,  # randomness of the sampling
                seed=777, # Seed for reprodicibility
                skip_special_tokens=False,  # Whether to skip special tokens in the output.
                max_tokens=2048,  # Maximum number of tokens to generate per output sequence.
            ),
            use_tqdm = True
        )
    else:
        print("Self Consistency not running!")
        responses = llm.generate(
            df["text"].values,
            vllm.SamplingParams(
                n=1,  # Number of output sequences to return for each prompt.
                top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
                temperature=0,  # randomness of the sampling
                seed=777, # Seed for reprodicibility
                skip_special_tokens=False,  # Whether to skip special tokens in the output.
                max_tokens=2048,  # Maximum number of tokens to generate per output sequence.
            ),
            use_tqdm = True
        )
    responses = [x.outputs[0].text for x in responses]
    df["fullLLMText"] = responses

    def extract_response(text):
        return ",".join(re.findall(r"<response>(.*?)</response>", text)).strip()

    responses = [extract_response(x) for x in responses]
    #responses_new_prompt = [extract_response(x) for x in responses_new_prompt]
    df["llmMisconception"] = responses
    #df_new_prompt["llmMisconception"] = responses
    if SELF_CONSISTENCY:
        if use_online_synthetic_data:
            df.to_parquet(f"/kaggle/working/submission.parquet", index=False)
        else:
            df.to_parquet(f"/kaggle/working/submission.parquet", index=False)
    else:
        if use_online_synthetic_data:
            df.to_parquet("/kaggle/working/submission.parquet", index=False)
        else:
            df.to_parquet(f"/kaggle/working/submission.parquet", index=False)
            print("File saved")
else:
    print("No reasoning executed!")
    df["fullLLMText"] = df["text"]
    df["llmMisconception"] = df["text"]
    if use_online_synthetic_data:
        df.to_parquet("/kaggle/working/submission.parquet", index=False)
    else:
        df.to_parquet("/kaggle/working/submission.parquet", index=False)

#provide question level data and label if needed
question_level_df = pd.DataFrame(question_level_df_data)
question_level_df.to_parquet("/kaggle/working/question_level_df.parquet", index=False)

Writing run_vllm.py


In [9]:
!python run_vllm.py

Executing: 
Entering test mode
Row question id:  1869
Row question id:  1870
Row question id:  1871
df_train head: 
   QuestionId  ConstructId  ...            AnswerCText             AnswerDText
0        1869          856  ...  \( 3 \times(2+4-5) \)  Does not need brackets
1        1870         1612  ...              \( m-1 \)       Does not simplify
2        1871         2774  ...     Both Tom and Katie      Neither is correct

[3 rows x 11 columns]
WARNING 12-11 13:38:12 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-11 13:38:12 config.py:715] Defaulting to use mp for distributed inference
INFO 12-11 13:38:12 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ', speculative_config=None, tokenizer='/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revi

In [10]:
"""llm_output = pd.read_parquet("/kaggle/working/submission.parquet")
print(llm_output.head())
if not IS_SUBMISSION:
    label = pd.read_parquet("/kaggle/working/label.parquet")
    print("Label: ")
    print(label.head())
    print("=====")
    label["MisconceptionReasoning"] = llm_output["llmMisconception"]

    print(label["MisconceptionReasoning"][0])
    label.to_parquet("temp.parquet", index=False)"""

'llm_output = pd.read_parquet("/kaggle/working/submission.parquet")\nprint(llm_output.head())\nif not IS_SUBMISSION:\n    label = pd.read_parquet("/kaggle/working/label.parquet")\n    print("Label: ")\n    print(label.head())\n    print("=====")\n    label["MisconceptionReasoning"] = llm_output["llmMisconception"]\n\n    print(label["MisconceptionReasoning"][0])\n    label.to_parquet("temp.parquet", index=False)'

## [](http://)Embedding Search


In [11]:
import pandas as pd
df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
print(df.head())

   QuestionId  ConstructId                                      ConstructName  \
0        1869          856  Use the order of operations to carry out calcu...   
1        1870         1612  Simplify an algebraic fraction by factorising ...   
2        1871         2774            Calculate the range from a list of data   

   SubjectId                                        SubjectName CorrectAnswer  \
0         33                                             BIDMAS             A   
1       1077                    Simplifying Algebraic Fractions             D   
2        339  Range and Interquartile Range from a List of Data             B   

                                        QuestionText            AnswerAText  \
0  \[\n3 \times 2+4-5\n\]\nWhere do the brackets ...  \( 3 \times(2+4)-5 \)   
1  Simplify the following, if possible: \( \frac{...              \( m+1 \)   
2  Tom and Katie are discussing the \( 5 \) plant...              Only\nTom   

              AnswerBText        

# Run finetuned embedding model search
Use finetuned SFR Mistral 2R to search for the nearest misconception

In [12]:
%%writefile embedding_search_setup.py
# Import Libraries
##########################################################################
import os
import re
import pandas as pd
import polars as pl
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from pylatexenc.latex2text import LatexNodes2Text
from peft import PeftModel
from datasets import Dataset
import torch
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from torch import Tensor

# Configurations
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

os.environ["TOKENIZERS_PARALLELISM"] = "false"
#clear cache
torch.cuda.empty_cache()
CONFIG = {
    "retrieve_num": 25,
    "data_path": "/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv",
    "model_name": "/kaggle/input/sfr-embedding-mistral/SFR-Embedding-2_R",
    "output_path": "/kaggle/working/submission.parquet",
    "target_column": "AllTextWithLlmMisconceptionCleaned",
    "checkpoint_path": "/kaggle/input/sfr-2-r-zeroshotselfconsistency-higherloradropout",
    "misconception_path": "/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv",
    "llm_answer_path": "/kaggle/working/submission.parquet",# have to change it
}

# Data Preparation
##########################################################################
def load_data():
    train = pl.read_csv(CONFIG['data_path'])
    misconception_mapping = pl.read_csv(CONFIG['misconception_path'])
    llm_answers = pl.read_parquet(CONFIG["llm_answer_path"])
    return train, misconception_mapping, llm_answers

def convert_to_long(df: pl.DataFrame,
                    common_cols: list =["QuestionId", "ConstructName", "SubjectName", "QuestionText", "CorrectAnswer"],
                   ):
    df_long = (
        df
        .select(
            pl.col(common_cols + [f"Answer{alpha}Text" for alpha in ["A", "B", "C", "D"]])
        )
        .unpivot(
            index=common_cols,
            variable_name="AnswerType",
            value_name="AnswerText",
        )
        .with_columns(
            pl.concat_str(
                [
                    pl.format("Construct Name:\n{}\n\n", pl.col("ConstructName")),
                    pl.format("Subject Name:\n{}\n\n", pl.col("SubjectName")),
                    pl.format("Question Text:\n{}\n\n", pl.col("QuestionText")),
                    pl.format("Answer Text:\n{}\n\n", pl.col("AnswerText")),
                ],
            separator=""
            ).alias("AllText"),
            pl.col("AnswerType").str.extract(r"Answer([A-D])Text$").alias("AnswerAlphabet"),
        )
        .with_columns(
            pl.concat_str(
                [pl.col("QuestionId"), pl.col("AnswerAlphabet")], separator="_"
            ).alias("QuestionId_Answer"),
        )
        .sort("QuestionId_Answer")
    )
    print("result in function")
    print(df_long.head())
    print("--------")
    return df_long

def add_llm_misconceptions(df_long: pl.DataFrame, llm_answers: pl.DataFrame) -> pl.DataFrame:
    df_merged = df_long.join(llm_answers[['QuestionId_Answer', 'llmMisconception']], on='QuestionId_Answer', how='left' )
    df_merged = df_merged.with_columns(pl.concat_str([pl.col('AllText'),pl.format("Misconception:\n{}", pl.col("llmMisconception"))]).alias(CONFIG["target_column"]))
    return df_merged

def preprocess_latex_column(df, column):
    def clean_latex(row):
        clean_text = LatexNodes2Text().latex_to_text(row[column])
        return re.sub(r":\n+\s*", ":\n", clean_text)

    return df.with_columns([
        pl.struct(pl.all()).map_elements(clean_latex, return_dtype=pl.String).alias(column)
    ])

train, misconception_mapping, llm_answers = load_data()
train_long = convert_to_long(train)
train_long = add_llm_misconceptions(train_long, llm_answers)
train_long = preprocess_latex_column(train_long, CONFIG["target_column"])
train_long = train_long.drop_nulls(CONFIG["target_column"])

# Load model
##########################################################################
from transformers import AutoModel

def load_model(checkpoint_path):
    """
    Load a model and distribute it across GPUs 1 and 2.

    Args:
        checkpoint_path (str): Path to the PEFT model checkpoint.

    Returns:
        model: The loaded model distributed across GPUs.
        tokenizer: The tokenizer corresponding to the model.
    """
    # Load the model with the device map
    model = AutoModel.from_pretrained(
        CONFIG["model_name"],
        torch_dtype=torch.bfloat16,
        device_map="balanced"
    )

    # Add PEFT layers
    model = PeftModel.from_pretrained(
        model,
        checkpoint_path,
        is_trainable=False,
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
    print(f"PEFT model loaded from '{checkpoint_path}' across GPUs 1 and 2.")

    return model, tokenizer

model, tokenizer = load_model(CONFIG["checkpoint_path"])
model.eval()

def prepare_queries_and_corpus(val_ds: Dataset, misconception_mapping: pd.DataFrame, target_column: str):
    """
    Prepare queries and corpus dictionaries for evaluator and inference.

    Args:
        val_ds (Dataset): Validation dataset.
        misconception_mapping (pd.DataFrame): Mapping of misconception IDs to names.
        target_column (str): Column name containing query text.

    Returns:
        tuple: (queries, corpus) dictionaries.
    """
    # Prepare corpus from misconception mapping
    corpus = dict(
        zip(misconception_mapping["MisconceptionId"], misconception_mapping["MisconceptionName"])
    )
    
    # Prepare queries from validation dataset
    queries = dict(
        zip(val_ds["QuestionId_Answer"], val_ds[target_column])
    )

    return queries, corpus

queries, corpus = prepare_queries_and_corpus(
    val_ds=train_long,
    misconception_mapping=misconception_mapping,
    target_column=CONFIG["target_column"]
)
print(f"Queries: {len(queries)}")
print(f"Corpus: {len(corpus)}")

# Sentece encoding
##########################################################################
# def encode_sentences(sentences, model, tokenizer, batch_size=64):
#     """
#     Encodes a list of sentences in batches using a model split across multiple GPUs.

#     Args:
#         sentences (list of str): The sentences to encode.
#         model: The Hugging Face transformer model distributed across GPUs.
#         tokenizer: The tokenizer corresponding to the model.
#         batch_size (int): The number of sentences to process per batch.

#     Returns:
#         torch.Tensor: The concatenated encoded sentence embeddings.
#     """
#     all_embeddings = []
#     model.eval()  # Ensure the model is in evaluation mode

#     # Get the device of the first layer from the device map
#     first_layer_device = next(iter(model.hf_device_map.values()))  # Get the device of the first layer

#     # Process sentences in batches
#     for i in range(0, len(sentences), batch_size):
#         batch_sentences = sentences[i:i + batch_size]

#         # Tokenize batch and send it to the first layer's device
#         inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt").to(first_layer_device)

#         # Forward pass
#         with torch.no_grad():
#             outputs = model(**inputs)

#         # Extract token embeddings
#         token_embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
#         attention_mask = inputs.attention_mask  # Shape: (batch_size, seq_len)

#         # Compute sentence embeddings by averaging token embeddings (masked)
#         input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
#         sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
#         sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
#         sentence_embeddings = sum_embeddings / sum_mask  # Shape: (batch_size, hidden_size)

#         # Collect embeddings on CPU
#         all_embeddings.append(sentence_embeddings)
#         print(f"Encoded batch {i}")
#     # Concatenate all embeddings
#     return torch.cat(all_embeddings, dim=0).cpu()

def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def encode_sentences(sentences, model, tokenizer, batch_size=64):
    """
    Encodes a list of sentences in batches using a model split across multiple GPUs.

    Args:
        sentences (list of str): The sentences to encode.
        model: The Hugging Face transformer model distributed across GPUs.
        tokenizer: The tokenizer corresponding to the model.
        batch_size (int): The number of sentences to process per batch.

    Returns:
        torch.Tensor: The concatenated encoded sentence embeddings.
    """
    all_embeddings = []
    model.eval()  # Ensure the model is in evaluation mode

    # Get the device of the first layer from the device map
    first_layer_device = next(iter(model.hf_device_map.values()))  # Get the device of the first layer

    # Process sentences in batches
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        # Tokenize batch and send it to the first layer's device
        inputs = tokenizer(batch_sentences, padding=True, truncation=True, max_length = 4098, return_tensors="pt").to(first_layer_device)

        # Forward pass
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract token embeddings
        embeddings = last_token_pool(outputs.last_hidden_state, inputs['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)  # Normalize embeddings

        # Collect embeddings on CPU
        all_embeddings.append(embeddings)
        print(f"Encoded batch {i}")
    # Concatenate all embeddings
    return torch.cat(all_embeddings, dim=0).cpu()

def perform_inference(model, tokenizer, queries, corpus, top_k=25, output_path="./submission.parquet", batch_size=64):
    if not queries or not corpus:
        raise ValueError("Queries or corpus cannot be empty.")

    # Step 1: Encode queries and corpus
    print("Encoding queries...")
    query_embeddings = encode_sentences(list(queries.values()), model, tokenizer, batch_size=batch_size)

    print("Encoding corpus...")
    corpus_embeddings = encode_sentences(list(corpus.values()), model, tokenizer, batch_size=batch_size)

    # Step 2: Compute cosine similarity on CPU
    print("Computing cosine similarity...")
    query_norm = torch.nn.functional.normalize(query_embeddings, p=2, dim=1)
    corpus_norm = torch.nn.functional.normalize(corpus_embeddings, p=2, dim=1)

    # Compute the cosine similarity
    cos_sim_matrix = torch.mm(query_norm, corpus_norm.T)
    #cos_sim_matrix = cosine_similarity(query_embeddings, corpus_embeddings.T)

    # Step 3: Sort indices to get top-k results
    print("Sorting results...")
    top_k_indices = torch.topk(cos_sim_matrix, k=top_k, dim=1).indices

    # Step 4: Prepare output
    output_data = []
    query_keys = list(queries.keys())
    corpus_keys = list(corpus.keys())

    for i, query_id in enumerate(query_keys):
        top_items = [corpus_keys[idx] for idx in top_k_indices[i].tolist()]
        output_data.append({"QuestionId_Answer": query_id, "MisconceptionId": " ".join(map(str, top_items))})

    # Step 5: Save results to parquet
    print(f"Saving results to {output_path}...")
    result_df = pd.DataFrame(output_data)
    result_df.to_parquet(output_path, index=False)
    print(f"Inference results saved to {output_path}.")

perform_inference(model, tokenizer, queries, corpus, top_k=CONFIG["retrieve_num"])

Writing embedding_search_setup.py


In [13]:
!python embedding_search_setup.py

result in function
shape: (5, 10)
┌─────────┬─────────┬─────────┬────────┬───┬────────┬────────┬────────┬────────┐
│ Questio ┆ Constru ┆ Subject ┆ Questi ┆ … ┆ Answer ┆ AllTex ┆ Answer ┆ Questi │
│ nId     ┆ ctName  ┆ Name    ┆ onText ┆   ┆ Text   ┆ t      ┆ Alphab ┆ onId_A │
│ ---     ┆ ---     ┆ ---     ┆ ---    ┆   ┆ ---    ┆ ---    ┆ et     ┆ nswer  │
│ i64     ┆ str     ┆ str     ┆ str    ┆   ┆ str    ┆ str    ┆ ---    ┆ ---    │
│         ┆         ┆         ┆        ┆   ┆        ┆        ┆ str    ┆ str    │
╞═════════╪═════════╪═════════╪════════╪═══╪════════╪════════╪════════╪════════╡
│ 1869    ┆ Use the ┆ BIDMAS  ┆ \[     ┆ … ┆ \( 3   ┆ Constr ┆ A      ┆ 1869_A │
│         ┆ order   ┆         ┆ 3      ┆   ┆ \times ┆ uct    ┆        ┆        │
│         ┆ of oper ┆         ┆ \times ┆   ┆ (2+4)- ┆ Name:  ┆        ┆        │
│         ┆ ations  ┆         ┆ 2+4-5  ┆   ┆ 5 \)   ┆ Use    ┆        ┆        │
│         ┆ to…     ┆         ┆ \]     ┆   ┆        ┆ the    ┆        ┆    

## Reranking Misconception
Use Qwen2.5-14b-instruct-awq survivorship algorithm to rerank top-k misconception to the first k position
Possible choices:
1. Not using the algorithm at all (`selection_method = "no_choose"`)
2. Use the algorithm (`selection_method = "no_choose"`), define the top k choice to be chosen in `top_k_choices` and the window for choice consideration as `m_window_option`. Aside from the requirements of `top_k_choices <= m_window_option`, note that the higher the top_k, the lower the window option, the better the result is, but will create more inference pass. To balance efficiency and performance boost, we use k=3, window_option=15

In [14]:
%%writefile run_output_selection.py

import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

#===== TUNE HERE ===========
m_window_option = 15
assert m_window_option <= 25
top_k_choices = 3
assert top_k_choices <= m_window_option
#model_path = "/flash2/aml/chenjiah24_wangwd24_lad24/Qwen2.5-7B-Instruct-AWQ"
selection_method = "choose_k_from_all_with_mwindow"
assert selection_method in ["no_choose", "choose_k_from_all_with_mwindow"]
model_path = "/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ"
similarity_result_path = "/kaggle/working/submission.parquet"
misconception_path = "/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv"
reference_parquet_path = "/kaggle/working/question_level_df.parquet"
#================================


tokenizer = AutoTokenizer.from_pretrained(model_path)
#convert pandas data into numpy for faster operation
similarity_search_df = pd.read_parquet(similarity_result_path)
#populate similarity_search_df with data from the train.csv
reference_df = pd.read_parquet(reference_parquet_path)
similarity_search_df = pd.merge(similarity_search_df, reference_df, on=['QuestionId_Answer'], how="left")
# Convert the 'MisconceptionId' column from a space-separated string to a list of integers
similarity_search_df['MisconceptionId'] = similarity_search_df['MisconceptionId'].apply(lambda x: list(map(int, x.split())))

# Convert the list of lists into a 2D NumPy array
indices = np.array(similarity_search_df['MisconceptionId'].tolist())
print(indices.shape)

def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
Pick the correct misconception number from the below:

{Retrival}
"""
# just directly give your answers.

def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"incorrect_answer"],
                    CorrectAnswer=row[f"correct_answer"],
                    Retrival=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text


misconception_df = pd.read_csv(misconception_path)

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()


def get_candidates(c_indices):
    candidates = []

    mis_names = misconception_df["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))
        
    return candidates


if selection_method == "no_choose":
    #no need to do anything, assuming the similarity search already do its job
    similarity_search_df = pd.read_parquet(similarity_result_path)
    similarity_search_df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)
elif selection_method in ["choose_k_from_all_with_mwindow"]:
    def compute_rounds(T, M, C):
        if M <= 0 or C <= 0:
            raise ValueError("M and C must be greater than 0.")
        if T <= C:
            return 0
        R = (T - C + M - 1) // M  # ceiling division for (T-C)/M
        return R

    # Example usage
    # Suppose indices is an N x T array of candidate IDs:
    T = indices.shape[1] 

    M = m_window_option  # Candidates per round
    C = top_k_choices   # Survivors chosen each round

    R = compute_rounds(T, M, C)
    print("Number of rounds (R):", R)

    # Initialize survivors with last C candidates
    survivors = indices[:, -C:]
    candidate_pool = indices[:, :-C]

    for i in range(R):
        # How many candidates this round?
        pick = min(M, candidate_pool.shape[1])
        
        # Extract the last 'pick' candidates from candidate_pool
        round_candidates = candidate_pool[:, -pick:]
        candidate_pool = candidate_pool[:, :-pick] if candidate_pool.shape[1] > pick else candidate_pool[:, :0] #else empty the candidate pool
        
        # Combine with current survivors
        c_indices = np.concatenate([round_candidates, survivors], axis=1)
        
        #empty the survivors
        survivors = np.zeros((indices.shape[0], 0), dtype=int)

        # Prepare for LLM inference
        for infer_round in range(C):
            similarity_search_df[f"retrieval"] = list(get_candidates(c_indices))  # Each row gets the array of candidates
            similarity_search_df["text"] = similarity_search_df.apply(lambda row: apply_template(row, tokenizer), axis=1)

        
            #each inference can only do 
            responses = llm.generate(
                similarity_search_df["text"].values,
                vllm.SamplingParams(
                    n=1,
                    top_k=1,
                    temperature=0,
                    seed=777,
                    skip_special_tokens=False,
                    max_tokens=1,
                    logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=[str(x) for x in range(1, c_indices.shape[1]+1)])]
                ),
                use_tqdm=True
            )
        
            responses = [x.outputs[0].text for x in responses]
            similarity_search_df["response"] = responses
            
            llm_choices = similarity_search_df["response"].astype(int).values - 1
            
            # Update survivors: pick the chosen candidate from each row
            new_survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)

            if infer_round < C - 1:
                # Step 1: Create a mask to filter out the chosen indices
                mask = np.ones(c_indices.shape, dtype=bool) # Start with a mask of all True values
                np.put_along_axis(mask, llm_choices[:, np.newaxis], False, axis=1)  # Set the chosen indices to False

                # Step 2: Apply the mask to filter out the chosen indices
                c_indices = np.array([row[mask_row] for row, mask_row in zip(c_indices, mask)])

            # Update survivors
            survivors = np.concatenate([survivors, new_survivors], axis=1)

    # After all rounds, 'survivors' holds the final chosen candidates
    # Create final MisconceptionId column
    results = []
    for i in range(indices.shape[0]):
        # survivors[i] is the chosen final candidate(s)
        final_surv = survivors[i]
        # All original candidates:
        all_cands = indices[i]
        chosen_set = list(final_surv) + [c for c in all_cands if c not in final_surv]
        results.append(" ".join(map(str, chosen_set)))

    similarity_search_df["MisconceptionId"] = results
    similarity_search_df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)
else:
    raise ValueError("SelectionMethod Undefined")

Writing run_output_selection.py


In [15]:
!python run_output_selection.py

(9, 25)
WARNING 12-11 13:51:45 config.py:246] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-11 13:51:45 config.py:715] Defaulting to use mp for distributed inference
INFO 12-11 13:51:45 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ', speculative_config=None, tokenizer='/kaggle/input/qwenqwen2-5-14b-instruct-awq/Qwen2.5-14B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=5120, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), obse

## Sanity

In [16]:
import pandas as pd
pd.read_csv("/kaggle/working/submission.csv")


,QuestionId_Answer,MisconceptionId
0,1869_B,2306 2488 2518 987 2221 1316 1999 2586 1672 19...
1,1869_C,2306 2488 2586 987 2221 1316 2518 1999 107 143...
2,1869_D,2306 2488 1672 987 2221 1999 1316 2586 1119 25...
3,1870_A,891 2398 2142 1593 1540 1469 2021 2307 59 2549...
4,1870_B,891 2398 59 1593 1540 2307 403 2021 1825 2549 ...
5,1870_C,891 2398 59 1593 1540 2307 403 1469 2021 1825 ...
6,1871_A,1073 1287 2319 2439 632 188 691 2211 397 1873 ...
7,1871_C,1287 1073 397 2439 632 2211 691 188 2319 1200 ...
8,1871_D,1287 1073 397 2439 632 2211 691 188 2319 1200 ...


In [1]:
"""IS_SUBMISSION = False
if not IS_SUBMISSION:
    import pandas as pd
    from eedi_metrics import mapk, averaged_recall_at_k, averaged_ndcg_at_k
    # Read the label and predicted data
    label = pd.read_parquet("label.parquet")
    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])
    original_predicted_no_llm = pd.read_csv("/flash2/aml/lad24/submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])

    # Read question identifiers for both predictions
    question_ids = pd.read_csv("submission.csv")["QuestionId_Answer"]
    question_ids_no_llm = pd.read_csv("/flash2/aml/lad24/submission.csv")["QuestionId_Answer"]

    # Filter and reorder labels for `predicted`
    filtered_label = label[label["QuestionId_Answer"].isin(question_ids)]
    filtered_label = filtered_label.set_index("QuestionId_Answer").loc[question_ids].reset_index()

    # Filter and reorder labels for `original_predicted_no_llm`
    filtered_label_no_llm = label[label["QuestionId_Answer"].isin(question_ids_no_llm)]
    filtered_label_no_llm = filtered_label_no_llm.set_index("QuestionId_Answer").loc[question_ids_no_llm].reset_index()

    # Calculate metrics
    print("Validation original MAP25: ", mapk(filtered_label_no_llm["MisconceptionId"].tolist(), original_predicted_no_llm))
    print("Validation MAP25: ", mapk(filtered_label["MisconceptionId"].tolist(), predicted))
    print("Validation original Recall25: ", averaged_recall_at_k(filtered_label_no_llm["MisconceptionId"].tolist(), original_predicted_no_llm))
    print("Validation Recall2525: ", averaged_recall_at_k(filtered_label["MisconceptionId"].tolist(), predicted))
    print("Validation original NDCG25: ", averaged_ndcg_at_k(filtered_label_no_llm["MisconceptionId"].tolist(), original_predicted_no_llm))
    print("Validation NDCG25: ", averaged_ndcg_at_k(filtered_label["MisconceptionId"].tolist(), predicted))"""

'IS_SUBMISSION = False\nif not IS_SUBMISSION:\n    import pandas as pd\n    from eedi_metrics import mapk, averaged_recall_at_k, averaged_ndcg_at_k\n    # Read the label and predicted data\n    label = pd.read_parquet("label.parquet")\n    predicted = pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])\n    original_predicted_no_llm = pd.read_csv("/flash2/aml/lad24/submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()])\n\n    # Read question identifiers for both predictions\n    question_ids = pd.read_csv("submission.csv")["QuestionId_Answer"]\n    question_ids_no_llm = pd.read_csv("/flash2/aml/lad24/submission.csv")["QuestionId_Answer"]\n\n    # Filter and reorder labels for `predicted`\n    filtered_label = label[label["QuestionId_Answer"].isin(question_ids)]\n    filtered_label = filtered_label.set_index("QuestionId_Answer").loc[question_ids].reset_index()\n\n    # Filter and reorder labels for `original_predicted_n